In [1]:
import pandas as pd
import numpy as np
pd.set_option('max_columns', 50)
from datetime import datetime

In [2]:
df = pd.read_csv('Data Set 2.csv')
df.head()

,OrderId,CustomerId,LocationId,channel,Amount,OrderDate,BillNo
0,35623100,6662,4416,foodpanda,267,1/1/2018,DEL
1,35622814,8237,4002,zomato,1051,1/1/2018,DEL
2,35623284,11912,4416,takeaway,714,1/1/2018,DEL
3,35631171,14563,218,zomato,834,1/1/2018,GGN
4,35635515,14833,295,call,544,1/1/2018,DEL


In [3]:
df = df[df['BillNo'] == 'GGN']

In [4]:
df['OrderDate']= pd.to_datetime(df['OrderDate'])

In [5]:
df['OrderPeriod'] = df.OrderDate.apply(lambda x: x.strftime('%Y-%m'))

In [6]:
df.head()

,OrderId,CustomerId,LocationId,channel,Amount,OrderDate,BillNo,OrderPeriod
3,35631171,14563,218,zomato,834,2018-01-01,GGN,2018-01
10,35634821,37698,192,foodpanda,687,2018-01-01,GGN,2018-01
15,35634847,47539,192,zomato,410,2018-01-01,GGN,2018-01
19,35630872,71209,218,zomato,2079,2018-01-01,GGN,2018-01
26,35630537,88127,218,zomato,530,2018-01-01,GGN,2018-01


In [7]:
#Revenue
RevByMonth = df.groupby('OrderPeriod')['Amount'].sum() 
RevByMonth = pd.DataFrame(RevByMonth)
RevByMonth.head()

,Amount
OrderPeriod,
2018-01,7166569
2018-02,6896659
2018-03,7112283
2018-04,7388678
2018-05,7526673


In [8]:
#Orders
OrderByMonth = df.groupby('OrderPeriod')['OrderId'].count() #Orders
OrderByMonth = pd.DataFrame(OrderByMonth)
OrderByMonth.head()

,OrderId
OrderPeriod,
2018-01,11359
2018-02,11134
2018-03,11858
2018-04,12735
2018-05,13238


In [9]:
#Customers
CusByMonth = df.groupby("OrderPeriod").agg({"CustomerId": pd.Series.nunique}) 
CusByMonth.head()

,CustomerId
OrderPeriod,
2018-01,8471
2018-02,8567
2018-03,8513
2018-04,9298
2018-05,9992


In [10]:
#Repeat Customers
grouped = df.groupby(['OrderPeriod', 'CustomerId'])
RC = grouped.agg({'OrderId': pd.Series.nunique})
RC.head()

OrderId
OrderPeriod CustomerId         
2018-01     180               1
            1001              1
            1322              1
            2590              1
            2698              1

In [11]:
RCNew = RC[RC.OrderId > 1]
RCNew.head()

OrderId
OrderPeriod CustomerId         
2018-01     6423              2
            7728              2
            14180             2
            29838             2
            37698             3

In [12]:
RepCustomers = RCNew.groupby('OrderPeriod').count() 
RepCustomers.rename(columns={'OrderId': 'RepeatCustomers'}, inplace=True)
RepCustomers.head()

,RepeatCustomers
OrderPeriod,
2018-01,1520
2018-02,1429
2018-03,1537
2018-04,1773
2018-05,1734


In [13]:
#New Customers
df.set_index('CustomerId', inplace=True)
df['CohortGroup'] = df.groupby(level=0)['OrderDate'].min().apply(lambda x: x.strftime('%Y-%m'))
df.reset_index(inplace=True)
df.head()

,CustomerId,OrderId,LocationId,channel,Amount,OrderDate,BillNo,OrderPeriod,CohortGroup
0,14563,35631171,218,zomato,834,2018-01-01,GGN,2018-01,2018-01
1,37698,35634821,192,foodpanda,687,2018-01-01,GGN,2018-01,2018-01
2,47539,35634847,192,zomato,410,2018-01-01,GGN,2018-01,2018-01
3,71209,35630872,218,zomato,2079,2018-01-01,GGN,2018-01,2018-01
4,88127,35630537,218,zomato,530,2018-01-01,GGN,2018-01,2018-01


In [14]:
grouped = df.groupby(['CohortGroup', 'OrderPeriod'])
newC = grouped.agg({'CustomerId': pd.Series.nunique,
                       'OrderId': pd.Series.nunique})
newC.head()

CustomerId  OrderId
CohortGroup OrderPeriod                     
2018-01     2018-01            8471    11359
            2018-02            2009     3552
            2018-03            1768     3095
            2018-04            1593     2714
            2018-05            1431     2388

In [15]:
def cohort_period(df):
    
    df['CohortPeriod'] = np.arange(len(df)) + 1
    return df

newC = newC.groupby(level=0).apply(cohort_period)
newC.head()

CustomerId  OrderId  CohortPeriod
CohortGroup OrderPeriod                                   
2018-01     2018-01            8471    11359             1
            2018-02            2009     3552             2
            2018-03            1768     3095             3
            2018-04            1593     2714             4
            2018-05            1431     2388             5

In [16]:
NewCustomers = newC[newC['CohortPeriod'] == 1]

In [17]:
NC = NewCustomers.reset_index()

In [18]:
NC = NC.drop(['CohortGroup'], axis =1)

In [19]:
NC = NC.set_index('OrderPeriod')

In [20]:
NC = NC[['CustomerId']]

In [21]:
NC.rename(columns={'CustomerId': 'NewCustomers'}, inplace=True)

In [22]:
NC.head()

,NewCustomers
OrderPeriod,
2018-01,8471
2018-02,6558
2018-03,5782
2018-04,5915
2018-05,6375


In [23]:
dfMerge = pd.concat([RevByMonth, OrderByMonth, CusByMonth, NC,RepCustomers],axis=1,sort=False).reset_index()
dfMerge.rename(columns = {'index':'OrderPeriod'})
dfMerge.rename(columns={'Amount': 'Revenue',
                        'OrderId': 'Orders',
                        'CustomerId':'Customers'}, inplace=True)

In [24]:
dfMerge['Frequency'] = dfMerge['Orders']/dfMerge['Customers']

In [25]:
dfMerge.head()

,OrderPeriod,Revenue,Orders,Customers,NewCustomers,RepeatCustomers,Frequency
0,2018-01,7166569,11359,8471,8471,1520,1.340928
1,2018-02,6896659,11134,8567,6558,1429,1.299638
2,2018-03,7112283,11858,8513,5782,1537,1.392928
3,2018-04,7388678,12735,9298,5915,1773,1.369649
4,2018-05,7526673,13238,9992,6375,1734,1.324860


In [26]:
dfMerge

,OrderPeriod,Revenue,Orders,Customers,NewCustomers,RepeatCustomers,Frequency
0,2018-01,7166569,11359,8471,8471,1520,1.340928
1,2018-02,6896659,11134,8567,6558,1429,1.299638
2,2018-03,7112283,11858,8513,5782,1537,1.392928
3,2018-04,7388678,12735,9298,5915,1773,1.369649
4,2018-05,7526673,13238,9992,6375,1734,1.324860
5,2018-06,7851698,13198,10121,6486,1747,1.304021
6,2018-07,9569768,15370,11638,7497,2263,1.320674
7,2018-08,7833780,12511,10051,5957,1535,1.244752
8,2018-09,8220624,18510,13634,9014,2825,1.357635
9,2018-10,10106259,18607,13566,8787,2754,1.371591


In [27]:
dfMerge = dfMerge.set_index('OrderPeriod')

In [28]:
dfMerge.transpose()

OrderPeriod,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04
Revenue,7.166569e+06,6.896659e+06,7.112283e+06,7.388678e+06,7.526673e+06,7.851698e+06,9.569768e+06,7.833780e+06,8.220624e+06,1.010626e+07,1.387711e+07,1.077088e+07,7.844187e+06,8.279877e+06,1.154804e+07,2.988059e+06
Orders,1.135900e+04,1.113400e+04,1.185800e+04,1.273500e+04,1.323800e+04,1.319800e+04,1.537000e+04,1.251100e+04,1.851000e+04,1.860700e+04,2.264100e+04,1.764800e+04,1.279200e+04,1.677300e+04,2.564300e+04,7.223000e+03
Customers,8.471000e+03,8.567000e+03,8.513000e+03,9.298000e+03,9.992000e+03,1.012100e+04,1.163800e+04,1.005100e+04,1.363400e+04,1.356600e+04,1.634000e+04,1.335400e+04,9.947000e+03,1.274400e+04,1.883000e+04,6.101000e+03
NewCustomers,8.471000e+03,6.558000e+03,5.782000e+03,5.915000e+03,6.375000e+03,6.486000e+03,7.497000e+03,5.957000e+03,9.014000e+03,8.787000e+03,1.008700e+04,7.234000e+03,4.902000e+03,6.598000e+03,1.050300e+04,2.857000e+03
RepeatCustomers,1.520000e+03,1.429000e+03,1.537000e+03,1.773000e+03,1.734000e+03,1.747000e+03,2.263000e+03,1.535000e+03,2.825000e+03,2.754000e+03,3.656000e+03,2.522000e+03,1.595000e+03,2.454000e+03,4.040000e+03,8.280000e+02
Frequency,1.340928e+00,1.299638e+00,1.392928e+00,1.369649e+00,1.324860e+00,1.304021e+00,1.320674e+00,1.244752e+00,1.357635e+00,1.371591e+00,1.385618e+00,1.321552e+00,1.286016e+00,1.316149e+00,1.361816e+00,1.183904e+00


In [29]:
dfMerge.to_excel(r'C:\Users\user\Desktop\Xeno\GGN Report.xlsx')